In [1]:
import pandas as pd
import numpy as np
from deepblast.trainer import LightningAligner
import matplotlib.pyplot as plt
import seaborn as sns

from deepblast.score import alignment_score
from deepblast.dataset.parse_mali import read_mali
from deepblast.dataset.utils import states2matrix
from deepblast.dataset.utils import state_f, tmstate_f, revstate_f

from Bio import SearchIO

%matplotlib inline

In [2]:
!ls ../data/structure_benchmarks

malidup  malisam


In [3]:
mali_root = '../data/structure_benchmarks/malidup'
manual = read_mali(mali_root, tool='manual')
s = list(manual[0].values) + list(manual[1].values) 
query = '\n'.join(list(manual.apply(lambda x: f">{x['query_id']}\n{x[0]}", axis=1).values))
hits = '\n'.join(list(manual.apply(lambda x: f">{x['hit_id']}\n{x[1]}", axis=1).values))
alns = query + hits
#open(f'{mali_root}/seqs.fasta', 'w').write(alns)

KeyError: 'query_id'

# Parse Blast

In [4]:
from deepblast.dataset.utils import state_f

def parse_blast_xml(blast_path):
    records = SearchIO.parse(blast_path, 'blast-xml')
    hit_list = []
    results = []
    for idx, cur in enumerate(records):
        for hit in cur.hits:
            for i, hsp in enumerate(hit.hsps):
                if cur.id != hit.id:
                    qs = hsp.fragment.query_start
                    qe = hsp.fragment.query_end
                    he = hsp.fragment.hit_end
                    hs = hsp.fragment.hit_start
                    query_s = str(hsp.fragment.query.seq)
                    hit_s = str(hsp.fragment.hit.seq)
                    aln_s = hsp.aln_annotation['similarity']
                    score = hsp.bitscore
                    expect = hsp.evalue
                    toks = list(map(str, [cur.id, hit.id, i, qs, qe, hs, he, query_s, hit_s, aln_s,
                                         score, expect]))
                    results.append(toks)
    columns = ['query_id', 'hit_id', 'fragment_num', 
               'query_start', 'query_end', 'hit_start', 'hit_end', 
               'query_string', 'hit_string', 'alignment_string', 'score', 'evalue']
    return pd.DataFrame(results, columns=columns)

def hit_argmax(x):
    i = np.argmin(x['evalue'])
    return x.iloc[i]

def get_blast_alignments(blast_path, mali_root):
    blast_df = parse_blast_xml(blast_path)
    # only consider the top scoring local alignments
    blast_df = blast_df.groupby(['query_id', 'hit_id']).apply(hit_argmax)
    manual = read_mali(mali_root, tool='manual')
    idx = set(map(tuple, manual[['query_id', 'hit_id']].values))
    bidx = set(list(blast_df.index))
    idx = list(set(idx) & set(bidx))
    blast_df = blast_df.loc[idx]
    
    states = list(map(state_f, zip(list(blast_df['query_string']), list(blast_df['hit_string']))))
    states = ''.join(list(map(revstate_f, states)))
    blast_df['aln'] = states
    
    return blast_df

In [5]:
res = get_blast_alignments(blast_path, mali_root)

NameError: name 'blast_path' is not defined

In [6]:
blast_path = '../results/blast/malidup_blast_alignments.xml'
blast_df = get_blast_alignments(blast_path)

TypeError: get_blast_alignments() missing 1 required positional argument: 'mali_root'

In [7]:
def hit_argmax(x):
    i = np.argmin(x['evalue'])
    return x.iloc[i]
blast_df = blast_df.groupby(['query_id', 'hit_id']).apply(hit_argmax)

NameError: name 'blast_df' is not defined

In [8]:
idx = set(map(tuple, manual[['query_id', 'hit_id']].values))
bidx = set(list(blast_df.index))
idx = list(set(idx) & set(bidx))

KeyError: "None of [Index(['query_id', 'hit_id'], dtype='object')] are in the [columns]"

In [ ]:
np.argmax(blast_df.fragment_num)

In [ ]:
blast_df.iloc[147]

In [ ]:
blast_df = blast_df.set_index(['query_id', 'hit_id'])

In [ ]:
blast_df.loc[('24', '167')]

In [ ]:
blast_df.loc[('24', '167')].query_string

# Parse HMMER

In [ ]:

def parse_hmmer_text(hmmer_path):
    records = SearchIO.parse(hmmer_path, 'hmmer3-text')
    hit_list = []
    results = []
    for idx, cur in enumerate(records):
        for hit in cur.hits:
            for i, hsp in enumerate(hit.hsps):
                if cur.id != hit.id:
                    qs = hsp.fragment.query_start
                    qe = hsp.fragment.query_end
                    he = hsp.fragment.hit_end
                    hs = hsp.fragment.hit_start
                    query_s = str(hsp.fragment.query.seq)
                    hit_s = str(hsp.fragment.hit.seq)
                    score = hsp.bitscore
                    expect = hsp.evalue
                    toks = list(map(str, [cur.id, hit.id, i, qs, qe, hs, he, query_s, hit_s,
                                         score, expect]))
                    results.append(toks)
    columns = ['query_id', 'hit_id', 'fragment_num',
               'query_start', 'query_end', 'hit_start', 'hit_end',
               'query_string', 'hit_string', 'score', 'evalue']
    return pd.DataFrame(results, columns=columns)


def hit_argmax(x):
    i = np.argmin(x['evalue'])
    return x.iloc[i]


def get_hmmer_alignments(hmmer_path, mali_root):
    hmmer_df = parse_hmmer_text(hmmer_path)
    # only consider the top scoring local alignments
    hmmer_df = hmmer_df.groupby(['query_id', 'hit_id']).apply(hit_argmax)
    manual = read_mali(mali_root, tool='manual')
    idx = set(map(tuple, manual[['query_id', 'hit_id']].values))
    bidx = set(list(hmmer_df.index))
    idx = list(set(idx) & set(bidx))
    hmmer_df = hmmer_df.loc[idx]

    states = list(map(state_f, zip(list(hmmer_df['query_string']), list(hmmer_df['hit_string']))))
    states = ''.join(list(map(revstate_f, states)))
    hmmer_df['aln'] = states
    return hmmer_df


In [ ]:
hmmer_path = '../results/hmmer/malidup-hmm.out'
hmmer_df = get_hmmer_alignments(hmmer_path, mali_root)

In [ ]:
hmmer_df

In [ ]:
results